In [8]:
import pandas as pd
from sequential.seq2pat import Seq2Pat, Attribute

# 1. CSVデータの読み込み
# CSVの構成: 1行が1回の購買履歴 (user_id, item, timestamp, store)
csv_file = "./sorted_interaction_base_data.csv" # 用意したファイル名に合わせてください
df = pd.read_csv(csv_file)

# ---------------------------------------------------------
# 2. データのソート（重要）
# Seq2Patで正しくパターン認識するために、順序を整えます。
# 第1キー: user_id (ユーザーごとにまとめる)
# 第2キー: timestamp (時系列順にする)
# 第3キー: item (同一時刻の場合は、辞書順/50音順に並べる)
# ---------------------------------------------------------
df_sorted = df.sort_values(
    by=["user_id", "timestamp", "item"],
    ascending=[True, True, True]
)

# 3. ユーザーごとにリスト化（集約）
# groupbyを使って、ユーザーIDごとに各カラムをリストに変換します
grouped = df_sorted.groupby('user_id').agg({
    'item': list,
    'timestamp': list,
    'store': list
})

# 4. Seq2Pat用のデータ抽出
# そのまま .tolist() するだけで「リストのリスト」になります
sequences = grouped['item'].tolist()
timestamps = grouped['timestamp'].tolist()
stores = grouped['store'].tolist()

# --- ここまででデータ準備完了 ---
# 1. Seq2Patオブジェクトの作成
seq2pat = Seq2Pat(sequences=sequences)

# 2. 属性（Attribute）の作成と制約の追加
# タイムスタンプ属性を作成
time_attr = Attribute(values=timestamps)


# 例: 買い物と買い物の間隔が 10 以下であること
#seq2pat.add_constraint(time_attr.gap() <= 10)

#必要であれば「間隔が1以上（同じ時間の連投は除く）」なども可能です
seq2pat.add_constraint(0 <= time_attr.gap() <= 0)

# 3. パターン抽出の実行
# min_frequency: そのパターンが出現するユーザー数の最小値
patterns = seq2pat.get_patterns(min_frequency=5)

print(f"発見されたパターン数: {len(patterns)}")
if len(patterns) > 0:
    print("--- 上位パターン例 ---")
    for p in patterns[:5]:
        print(p)

発見されたパターン数: 35
--- 上位パターン例 ---
['パスタ', '緑茶', 7]
['マスク', 'ワイン', 7]
['ウイスキー', 'コーラ', 6]
['エナジードリンク', 'クッキー', 6]
['エナジードリンク', 'サンドイッチ', 6]


In [ ]:
# 1. Seq2Patの初期化
seq2pat = Seq2Pat(sequences=sequences)

# 2. 属性（Attribute）の定義
# タイムスタンプを使って、「どれくらいの間隔で買ったか」を制約にします
time_attr = Attribute(values=timestamps)

# -----------------------------------------------------------------------
# 【重要】離脱予測・スイッチ分析のための制約設定
# -----------------------------------------------------------------------

# 制約A: 時間間隔 (Gap Constraint)
# 「数ヶ月ぶりの購入」ではなく、「習慣的な購買の流れ」を見たいので、
# アイテム間の間隔が「5単位以内（例: 短期間）」であるものに限定します。
seq2pat.add_constraint(time_attr.gap() <= 5)

# 3. パターンの抽出実行
# min_frequency: データ量が少ない場合は 2~3、多い場合はユーザー数の 1% 程度に設定
# ここではサンプルデータなので「2人以上に出現」するパターンを探します
patterns = seq2pat.get_patterns(min_frequency=2)

# -----------------------------------------------------------------------
# 4. 結果の整形と「ビール」関連パターンのフィルタリング
# -----------------------------------------------------------------------
print(f"全発見パターン数: {len(patterns)}")

# ビールを含むパターンだけを抽出して表示（これが離脱分析の鍵になります）
beer_patterns = [p for p in patterns if "ビール" in p]

print("\n--- ビールに関連する購買パターン (Top 10) ---")
# パターンは (sequence, frequency) のタプルで返ってきます
# frequency（出現数）が多い順にソートして表示
beer_patterns.sort(key=lambda x: x[-1], reverse=True)

print(beer_patterns)
for row in beer_patterns[:10]:
    # *pat で「最後の要素以外すべて」を受け取り、freq で「最後の要素」を受け取ります
    *pat, freq = row
    
    print(f"{freq}人のユーザー: {pat}")
# -----------------------------------------------------------------------
# 5. 特徴量化への応用イメージ
# 「ビール -> チューハイ」のようなスイッチパターンがあるか確認
# -----------------------------------------------------------------------
print("\n--- 要注意パターン（スイッチ/離脱の予兆）の探索例 ---")
switch_candidates = [
    p for p in beer_patterns 
    if len(p) >= 2         # 長さが2以上（連鎖している）
    and p[0] == "ビール"    # 最初がビールで...
]

if switch_candidates:
    for row in switch_candidates:
        *pat, freq = row
        print(f"スイッチの可能性あり ({freq}人): {pat}")
else:
    print("今回のサンプルデータでは明確なスイッチパターンは見つかりませんでした。")

全発見パターン数: 2708

--- ビールに関連する購買パターン (Top 10) ---
[['ビール', '緑茶', 4], ['ビール', '緑茶', 'ポテトチップス', 2], ['ビール', '紅茶', 5], ['ビール', '紅茶', '緑茶', 3], ['ビール', '紅茶', 'チューハイ', 2], ['ビール', '紅茶', 'チューハイ', 'ナッツ', 2], ['ビール', '紅茶', 'チューハイ', 'ナッツ', '紅茶', 2], ['ビール', '紅茶', 'チューハイ', '紅茶', 2], ['ビール', '紅茶', 'ナッツ', 2], ['ビール', '紅茶', 'ナッツ', 'チューハイ', 2], ['ビール', '紅茶', 'ナッツ', 'チューハイ', 'ポテトチップス', 2], ['ビール', '紅茶', 'ナッツ', 'チューハイ', '紅茶', 2], ['ビール', '紅茶', 'ナッツ', '紅茶', 2], ['ビール', '紅茶', 'ポテトチップス', 2], ['ビール', '紅茶', '緑茶', 'ポテトチップス', 2], ['ビール', '焼酎', 3], ['ビール', '焼酎', 'ウイスキー', 2], ['ビール', '焼酎', 'チューハイ', 2], ['ビール', '洗剤', 9], ['ビール', '洗剤', 'ポテトチップス', 3], ['ビール', '洗剤', '緑茶', 3], ['ビール', '洗剤', 'クッキー', 2], ['ビール', '洗剤', 'コーラ', 2], ['ビール', '洗剤', 'コーラ', 'ポテトチップス', 2], ['ビール', '洗剤', 'コーラ', '焼酎', 2], ['ビール', '洗剤', 'パスタ', 2], ['ビール', '洗剤', '焼酎', 2], ['ビール', '洗剤', '焼酎', 'チューハイ', 2], ['ビール', '洗剤', '紅茶', 2], ['ビール', '歯磨き粉', 6], ['ビール', '歯磨き粉', '洗剤', 3], ['ビール', '歯磨き粉', '緑茶', 3], ['チューハイ', '歯磨き粉', 'ナッツ', 'ビール', 2], ['チューハイ', '歯磨き